# Lightweight Fine-Tuning Project

* PEFT technique: Progressive Embedding Fine-Tuning (PEFT)
* Model: GPT-2
* Evaluation approach: Train-validation split, followed by evaluation on a separate test set
* Fine-tuning dataset: Wikitext-103

# Import Libraries

In [ ]:
!pip install -q numpy pandas datasets transformers scikit-learn torch peft

In [ ]:
import torch

import numpy as np
import pandas as pd

from datasets import load_dataset

from transformers import AutoModelForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, Trainer, TrainingArguments

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model

In [ ]:
!pip freeze > requirements.txt

# Auxiliar Functions

In [ ]:
def tokenize(input):
    return tokenizer(input["text"], truncation=True, padding=True)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1) if isinstance(predictions, np.ndarray) else np.argmax(predictions, axis=1)

    return {
        "accuracy": accuracy_score(labels, predictions),
        "precision": precision_score(labels, predictions),
        "recall": recall_score(labels, predictions),
        "f1": f1_score(labels, predictions),
    }


# Load Dataset

In [ ]:
dataset = load_dataset("wikitext", "wikitext-103-raw-v1")

train_ds = dataset["train"]
test_ds = dataset["test"]
validation_ds = dataset["validation"]

## Example

In [ ]:
train_ds["text"][:5]

# Loading and Evaluating a Foundation Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "[PAD]"})

model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

In [ ]:
preprocessed_train_ds = train_ds.map(tokenize, batched=True)
preprocessed_test_ds = test_ds.map(tokenize, batched=True)
preprocessed_validation_ds = validation_ds.map(tokenize, batched=True)

In [ ]:
preprocessed_train_ds["text"][:5]

In [ ]:
preprocessed_train_ds

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_dir="./logs",
    per_device_eval_batch_size=16,
    report_to="tensorboard",
    num_train_epochs=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
evaluation_result = trainer.evaluate(preprocessed_validation_ds)

evaluation_result

In [ ]:
inputs = tokenizer("Hello, my name is ", return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)

tokenizer.batch_decode(outputs)

# Performing Parameter-Efficient Fine-Tuning

In [ ]:
config = LoraConfig()

In [ ]:
lora_model = get_peft_model(model, config).to(device)

lora_model.print_trainable_parameters()

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    report_to="tensorboard",
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=preprocessed_train_ds,
    eval_dataset=preprocessed_validation_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
results = trainer.evaluate(eval_dataset=preprocessed_test_ds)

In [ ]:
lora_model.save_pretrained("gpt-lora")

# Performing Inference with a PEFT Model

In [ ]:
lora_model = AutoPeftModelForCausalLM.from_pretrained("gpt-lora")

In [ ]:
inputs = tokenizer("Hello, my name is ", return_tensors="pt")
outputs = lora_model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)

tokenizer.batch_decode(outputs)